In [6]:
#Importa os pacotes necessários

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
plt.style.use('dark_background')

#### Dicionário de Variáveis utilizadas
|Chave|Descrição|Função|
|-----|-----|-----|
|Ano| Ano de referência|Identificação|
|Trimestre| Trimestre de referência|Identificação|
|UF| Estado|Identificação|
|Capital| Capital da UF|Identificação|
|UPA| Unidade Primária de Amostragem (UF(2) + número squencial(6) + DV(1))|
|V1008| Número do Domicílio|Identificação|
|Estrato||
|V1014| Painel - Grupo de Amostra|
|V1016| Número de entrevista (1 a 5)|
|V2003| Número de ordem|
|V2007| Sexo|Identificação|
|V2008| Dia Nascimento|
|V20081| Mês nascimento|
|V20082| Ano nascimento|
|V2010| Raça|Identificação|
|V4041| Código de Ocupação -> cruzar com a tabela COD 2010|Medida|
|V4044| Código da Principal atividade da empresa -> cruzar a CNAE-Domicilar|Medida|
|V1028| Peso do domicílio e das pessoas sem calibração|Expansão|
|VD1029| Projeção da população|Expansão|

Chave para pessoa: UPA + V1008 + V1014  + V2003 + V2007 + V2008 + V20081 + V20082

**PARA ESTIMAÇÃO DA POPULAÇÃO, DEVE-SE USAR AS VARIÁVEIS: V1033 e posest.**

#### Importação e tratamento dos microdados

Importação dos microdados

In [7]:
#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\'
else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/'



#Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

colunas_selecionadas = ['Ano', 'Trimestre', 'UPA', 'UF', 'V1008', 'V1014', 'V2007', 'V2008', 'V20081', 'V20082', 'V2010', 'V4041', 'V4044', 'V1028', 'V1029', 'V1033', 'posest', 'posest_sxi']

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding='latin-1')
        if 2019 < ano < 2022:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding='latin-1')
        if ano == 2022 and tri < 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding='latin-1')

2019 4
2020 1
2020 2
2020 3
2020 4
2021 1
2021 2
2021 3
2021 4
2022 1
2022 2


In [8]:
#Conversão das variável V2007(sexo) para inteira para criar ID do indivíduo. (Homem = 0, Mulher = 1)
for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)


In [9]:
#Organização das variáveis e criação da chave de ID do indivíduo em cada dataframe.

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })

In [10]:
#Concatena todos os dataframes de cada ano e trimestre em uma unica tabela.

pnadc_completa = pd.concat([pnad_2019_4_id_cnae, 
                            pnad_2020_1_id_cnae, 
                            pnad_2020_2_id_cnae, 
                            pnad_2020_3_id_cnae, 
                            pnad_2020_4_id_cnae, 
                            pnad_2021_1_id_cnae, 
                            pnad_2021_2_id_cnae, 
                            pnad_2021_3_id_cnae, 
                            pnad_2021_4_id_cnae, 
                            pnad_2022_1_id_cnae, 
                            pnad_2022_2_id_cnae])

In [11]:
pnadc_completa

,Ano,Trimestre,UPA,UF,V2007,ID,V2010,V4041,V4044,V1028,V1029,V1033,posest,posest_sxi
0,2019,4,110000016,Rondônia,1,110002003,Parda,NaN,NaN,145.628790,532471,6982307,111,210
1,2019,4,110000016,Rondônia,1,110002034,Parda,NaN,NaN,145.628790,532471,8485527,111,206
2,2019,4,110000016,Rondônia,1,110002051,Parda,NaN,NaN,145.628790,532471,9316281,111,204
3,2019,4,110000016,Rondônia,0,110002077,Parda,NaN,NaN,145.628790,532471,7432741,111,102
4,2019,4,110000016,Rondônia,0,110002012,Parda,NaN,NaN,134.017538,532471,5214819,111,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482113,2022,2,530057544,Distrito Federal,0,530059585,Parda,NaN,NaN,81.626590,3125014,8442398,531,107
482114,2022,2,530057544,Distrito Federal,0,530059543,Preta,NaN,NaN,72.323502,3125014,3560554,531,114
482115,2022,2,530057544,Distrito Federal,0,530059578,Preta,NaN,NaN,72.323502,3125014,8409440,531,108
482116,2022,2,530057544,Distrito Federal,1,530059586,Branca,NaN,NaN,72.323502,3125014,8558427,531,207


In [12]:
# Ordernação da série histórica por ano e trimestre.
pnadc_completa.sort_values(by=['Ano', 'Trimestre'], inplace=True)

In [13]:
#concatena a coluna 'Ano' e 'Trimestre' em uma única coluna.
pnadc_completa['ano_tri'] = pnadc_completa['Ano'].astype(str) + 'Q' + pnadc_completa['Trimestre'].astype(str)

In [14]:
#Conversão da coluna ano_tri para datetime.
pnadc_completa['ano_tri'] = pd.PeriodIndex(pnadc_completa['ano_tri'], freq='Q')

#### Cruzamento da CNAE domiciliar com a base da PNAD  para identificar o setor de atividade econômica.

In [47]:
cod_2010 = pd.read_excel('Anexos\COD 2010.xlsx')

In [48]:
cod_2010

,Código,Grupo,Subgrupo,Subgrupo.1,Grupo de classe,Titulação
0,1,1,NaN,NaN,NaN,DIRETORES E GERENTES
1,11,1,11.0,NaN,NaN,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."
2,111,1,11.0,111.0,NaN,MEMBROS SUPERIORES DO PODER EXECUTIVO E LEGISL...
3,1111,1,11.0,111.0,1111.0,LEGISLADORES
4,1112,1,11.0,111.0,1112.0,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
...,...,...,...,...,...,...
198,3142,3,31.0,314.0,3142.0,TÉCNICOS AGROPECUÁRIOS
199,3143,3,31.0,314.0,3143.0,TÉCNICOS FLORESTAIS
200,315,3,31.0,315.0,NaN,TÉCNICOS E CONTROLADORES DA NAVEGAÇÃO MARÍTIMA...
201,3151,3,31.0,315.0,3151.0,OFICIAIS MAQUINISTAS EM NAVEGAÇÃO


In [49]:
cod_2010['Subgrupo'].value_counts()

21.0    31
31.0    29
26.0    27
22.0    22
13.0    19
23.0    18
24.0    15
25.0    12
12.0    10
14.0     9
11.0     8
Name: Subgrupo, dtype: int64

In [21]:
pnadc_completa['V4041'].value_counts()

2341.0    3923
6121.0    3450
6111.0    3383
5221.0    2286
2330.0    2086
          ... 
8132.0       1
8143.0       1
7535.0       1
3433.0       1
9312.0       1
Name: V4041, Length: 388, dtype: int64